#Utils

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive/workspace/dl_camera_pointer 

##Camera input functions

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

#Camera Capture

In [ ]:
counter = 111

In [ ]:
from IPython.display import Image
try:
  filename = take_photo(f"images/sample_{counter}.png")
  print('Saved to {}'.format(filename))
  counter += 1
  display(Image(filename))
except Exception as err:
  print(str(err))

#Labeling

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
import pandas as pd

def labelPhotos(images_b64, points_to_anotate, output_path=None):
  js = Javascript('''
      async function labeler(images_b64, points_to_anotate) {
        var k_element = 0;
        var i_point = 1;

        const buttons_div = document.createElement('div');
        
        const image_title = document.createElement('h2');
        image_title.innerHTML = "sample_"+k_element+".png";
        buttons_div.appendChild(image_title);

        const previous = document.createElement('button');
        previous.textContent = 'Previous';
        buttons_div.appendChild(previous);

        const next = document.createElement('button');
        next.textContent = 'Next';
        buttons_div.appendChild(next);

        const clear = document.createElement('button');
        clear.textContent = 'Clear';
        buttons_div.appendChild(clear);

        const finish_button = document.createElement('button');
        finish_button.textContent = 'Save data!';
        buttons_div.appendChild(finish_button);
        
        const debug_txt = document.createElement('p');
        debug_txt.innerHTML = "You can see here the coordinates";
        buttons_div.appendChild(debug_txt);

        document.body.appendChild(buttons_div);

        var new_annotation = ()=>{
          annotation = {"is_any_point": 0};
          for (let i = 1; i <= points_to_anotate; i++) {
            annotation["pp_" + i] = 0;
            annotation["x" + i] = null;
            annotation["y" + i] = null;
            annotation["r" + i] = null;
          }
          return annotation;
        }

        var annotations = [];
        for (let k = 0; k < images_b64.length; k++) {
          annotations.push(new_annotation());
        }

        const size = {width: 640, height: 480};
        const div_container = document.createElement('div');
        div_container.style.width=size.width;
        div_container.style.height=size.height;
        div_container.style.display="inline-block";

        const image_disp = new Image();
        image_disp.src = 'data:image/png;base64,'+images_b64[k_element].slice(2, -1);
        image_disp.style.position = "absolute";
        image_disp.style.zIndex = 1;
        div_container.appendChild(image_disp);

        const canvas = document.createElement('canvas');
        canvas.width = size.width;
        canvas.height =size.height;
        canvas.style.borderStyle = "solid";
        canvas.style.display = 'block';
        canvas.style.position = "relative";
        canvas.style.zIndex = 20;

        const initial_radius = 7;
        var drawCircle = (canvas, x, y, radius=initial_radius)=>{
          var context = canvas.getContext("2d");
          context.beginPath();
          context.arc(x, y, radius, 0, 2 * Math.PI);
          context.fillStyle = 'red';
          context.fill();
          context.lineWidth = 3;
          context.strokeStyle = '#000000';
          context.stroke();
        };

        var reDrawCanvas = (canvas)=>{
          canvas.getContext("2d").clearRect(0, 0, canvas.width, canvas.height);
          for (let i = 1; i <= points_to_anotate; i++) {
            if (annotations[k_element]["x"+i] != null){
              drawCircle(canvas, annotations[k_element]["x"+i], annotations[k_element]["y"+i], annotations[k_element]["r"+i]);
            }
          }
        };

        div_container.appendChild(canvas);
        document.body.appendChild(div_container);
        
        var mouse = {x: 0, y: 0};
        var updateMouse = (e) => {
          mouse.x = e.pageX - canvas.offsetLeft;
          mouse.y = e.pageY - canvas.offsetTop;
          debug_txt.innerHTML = "x: "+ (mouse.x) + "y: " + (mouse.y);
        }
        
        var move_circle = (e) =>{
          updateMouse(e);
          reDrawCanvas(canvas);
          drawCircle(canvas, mouse.x, mouse.y, initial_radius);
        };

        var radius = 0;
        var resize_circle = (e) =>{
          updateMouse(e);
          reDrawCanvas(canvas);
          radius = ((circle.x - mouse.x)**2 + (circle.y - mouse.y)**2)**(1/2);
          drawCircle(canvas, circle.x, circle.y, radius);
        };
        
        var stop_resizing_point = (e) =>{
          annotations[k_element]["is_any_point"] = 1;
          annotations[k_element]["pp_"+i_point] = 1;
          annotations[k_element]["x"+i_point] = circle.x;
          annotations[k_element]["y"+i_point] = circle.y;
          annotations[k_element]["r"+i_point] = radius;

          canvas.removeEventListener('mousemove', resize_circle);
          canvas.removeEventListener('mousedown', stop_resizing_point);
          i_point += 1 
          if (i_point <= points_to_anotate) {
            canvas.addEventListener('mousemove', move_circle);
            canvas.addEventListener('mousedown', stop_moving_point);
          }
        }

        var stop_moving_point = (e) =>{
          circle = {x: mouse.x, y: mouse.y, radius: initial_radius};
          radius = circle.radius;

          canvas.removeEventListener('mousemove', move_circle);
          canvas.removeEventListener('mousedown', stop_moving_point);
          canvas.addEventListener('mousemove', resize_circle);
          canvas.addEventListener('mousedown', stop_resizing_point);
        }

        var mod_func = (n, m) =>{
          return ((n % m) + m) % m
        }

        var start_new_image = (dir) =>{
          i_point = 1;
          k_element = mod_func((k_element + dir), images_b64.length);
          image_title.innerHTML = "sample_"+k_element+".png";
          image_disp.src = 'data:image/png;base64,'+images_b64[k_element].slice(2, -1);
          reDrawCanvas(canvas);
          canvas.addEventListener('mousemove', move_circle);
          canvas.addEventListener('mousedown', stop_moving_point);
        }

        start_new_image(null)
        previous.addEventListener('click', function (e) {
          start_new_image(-1)
        });
        next.addEventListener('click', function (e) {
          start_new_image(1)
        });
        clear.addEventListener('click', function (e) {
          annotations[k_element] = new_annotation()
          start_new_image(0)
        });

        await new Promise((resolve) => finish_button.onclick = resolve);
        return annotations
      }
      ''')

  display(js)
  data = eval_js('labeler({}, {})'.format(images_b64, points_to_anotate))
  return pd.DataFrame(data)

In [ ]:
import os
from IPython.display import Image
from PIL import Image
import base64
from io import BytesIO

image_folder = "images/"
images = os.listdir(image_folder)
images_str = None
for file in images:
  image = Image.open(f"{image_folder}{file}")
  buffered = BytesIO()
  image.save(buffered, format="JPEG")
  img_str = base64.b64encode(buffered.getvalue())
  if images_str:
    images_str += f', "{img_str}"'
  else:
    images_str = f'"{img_str}"'

In [ ]:
version = 1
labeled_data = labelPhotos(f'[{images_str}]', 2)
labeled_data.to_csv(f"labeled_data/data_v{version}.csv") #TODO: LOAD PREV DATA VERSIONS
labeled_data